<a name="Data"></a>
### DATA
This section will unload all of the .npz info I had saved and the axial, saggital, and coronal slices to create individual data entries.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Fri Feb  7 14:44:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             42W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# Unload all data such that it's easily accessible
main_data_path = "/content/drive/MyDrive/ENGSCI/4TH YEAR/fall 4th year/ESC499/Code Test/Finetuning/Data/"
training_data_indices = main_data_path+"INDICES.npz"
training_data_path = main_data_path+"training"
test_data_path = main_data_path+"test"

In [ ]:
# Import libraries
import numpy as np
import os
from PIL import Image
import random

random.seed(42)

In [ ]:
# Functions

def load_scan_from_npz(file_path):
    data = np.load(file_path)
    return data['voxel'], data['ax'], data['sag'], data['cor'], data['label']

def create_image_dataset(file_path):
    v, a, s, c, l = load_scan_from_npz(file_path)
    begin, end = a
    image_label_pairs = []
    for i in range(begin+20, end+1-20, 1):
        np_array, label = v[i], l
        np_array = np.uint8(255 * (np_array - np.min(np_array)) / (np.max(np_array) - np.min(np_array)))
        image = Image.fromarray(np_array)
        image_label_pairs.append({"image":image, "label":label})
    return image_label_pairs

def counts_zeros_and_ones(full_list):
    zeros = [entry for entry in full_list if entry["label"] == 0]
    ones = [entry for entry in full_list if entry["label"] == 1]
    num_zeros, num_ones = len(zeros), len(ones)
    print(f"Number of ZEROS: {len(zeros)} || Number of ONES: {len(ones)}")
    return num_zeros, num_ones, zeros, ones

def oversample_dataset(full_list):
    num_zeros, num_ones, zeros, ones = counts_zeros_and_ones(full_list)
    if num_zeros < num_ones:
        oversampled_zeros = random.choices(zeros, k=num_ones)
        balanced_list = oversampled_zeros + ones
    else:
        oversampled_ones = random.choices(ones, k=num_zeros)
        balanced_list = zeros + oversampled_ones
    random.shuffle(balanced_list)
    return balanced_list

def convert_to_conversation(sample):
    instruction = "Classify the brain scan as Low Grade Glioma (0), High Grade Glioma (1), or No Glioma (2). Respond only in the following format: Choice: <0, 1, or 2> Reasoning: <Provide concise reasoning using 10 keywords based on the scan's visual features>."

    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : sample['image']} ]
        },
        { "role" : "assistant",
          "content" : [
            {"type": "text", "text": f"Choice: {sample['label']} Reasoning:..."} ]
        },
    ]
    return { "messages" : conversation }

def generate_ratio_vector(filenames, ratio):
    num_files = len(filenames)
    num_ones = int(num_files * ratio)
    vector = [1] * num_ones + [0] * (num_files - num_ones)
    np.random.shuffle(vector)
    return vector

# def create_conversation_dataset(training_data_path):

#     # Extract all the patients and the corresponding filenames
#     filenames = []
#     for filename in os.listdir(training_data_path):
#         if filename.endswith(".npz"):
#             file_path = os.path.join(training_data_path, filename)
#             filenames.append(file_path)
#     filenames = sorted(filenames, key=lambda x: int(x.split('_')[-1].split('.')[0]))

#     # Now we build the dataset and balance it
#     dataset = []
#     for filename in filenames:
#         image_label_pairs = create_image_dataset(filename)
#         dataset = dataset + image_label_pairs
#         # print(f"Added {len(image_label_pairs)} in {filename.split('/')[-1]}")
#     balanced_dataset = oversample_dataset(dataset)
#     counts_zeros_and_ones(balanced_dataset)

#     # # Now convert the dataset into input for LLM
#     converted_dataset = [convert_to_conversation(sample) for sample in balanced_dataset]
#     print(f"Dataset size: {len(converted_dataset)}")
#     print(converted_dataset[0])

#     return balanced_dataset, converted_dataset

def create_conversation_dataset(training_data_path):

    # Extract all the patients and the corresponding filenames
    filenames = []
    for filename in os.listdir(training_data_path):
        if filename.endswith(".npz"):
            file_path = os.path.join(training_data_path, filename)
            filenames.append(file_path)
    filenames = sorted(filenames, key=lambda x: int(x.split('_')[-1].split('.')[0]))

    # Now we build the dataset and balance it
    train_dataset, val_dataset = [], []
    vector = generate_ratio_vector(filenames, 0.8)
    for filename in filenames:
        image_label_pairs = create_image_dataset(filename)
        list_add = vector.pop()
        if list_add == 1: train_dataset = train_dataset + image_label_pairs
        else: val_dataset = val_dataset + image_label_pairs

    print("Training Data")
    balanced_train_dataset = oversample_dataset(train_dataset)
    counts_zeros_and_ones(balanced_train_dataset)

    print("Validation Data")
    balanced_val_dataset = oversample_dataset(val_dataset)
    counts_zeros_and_ones(balanced_val_dataset)

    # Now convert the dataset into input for LLM
    converted_train_dataset = [convert_to_conversation(sample) for sample in balanced_train_dataset]
    print(f"Dataset size: {len(converted_train_dataset)}")
    print(converted_train_dataset[0])
    converted_val_dataset = [convert_to_conversation(sample) for sample in balanced_val_dataset]
    print(f"Dataset size: {len(converted_val_dataset)}")
    print(converted_val_dataset[0])

    return balanced_train_dataset, balanced_val_dataset, converted_train_dataset, converted_val_dataset






balanced_train_dataset, balanced_val_dataset, converted_train_dataset, converted_val_dataset = create_conversation_dataset(training_data_path)
# dataset, converted_dataset = create_conversation_dataset(training_data_path)

Training Data
Number of ZEROS: 1114 || Number of ONES: 4564
Number of ZEROS: 4564 || Number of ONES: 4564
Validation Data
Number of ZEROS: 315 || Number of ONES: 1125
Number of ZEROS: 1125 || Number of ONES: 1125
Dataset size: 9128
{'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': "Classify the brain scan as Low Grade Glioma (0), High Grade Glioma (1), or No Glioma (2). Respond only in the following format: Choice: <0, 1, or 2> Reasoning: <Provide concise reasoning using 10 keywords based on the scan's visual features>."}, {'type': 'image', 'image': <PIL.Image.Image image mode=L size=128x128 at 0x7DBD36A09010>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': 'Choice: 0 Reasoning:...'}]}]}
Dataset size: 2250
{'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': "Classify the brain scan as Low Grade Glioma (0), High Grade Glioma (1), or No Glioma (2). Respond only in the following format: Choice: <0, 1, or 2> Reasoning: <Provide concise reaso

<a name="LLM Setup"></a>
### LLM SETUP
This notebook finetunes **Llama 3.2 11B Vision Instruct** to (hopefully) better detect Low VS High Grade Gliomas from Flair Torso Scans.

In [ ]:
# 42s 44s
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
# 44s
!pip install --upgrade torchvision
import torchvision
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig
from transformers import TextStreamer
from transformers import TrainerCallback, TrainingArguments

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.9 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu124
    Uninstalling torchvision-0.20.1+cu124:
      Successfully uninstalled torchvision-0.20.1+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.18 requires torch<2.6,>=1.10, but you have torch 2.6.0 which is incompatible.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit", # Llama 3.2 vision support
    "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
    "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit", # Can fit in a 80GB card!
    "unsloth/Llama-3.2-90B-Vision-bnb-4bit",
    "unsloth/Pixtral-12B-2409-bnb-4bit",              # Pixtral fits in 16GB!
    "unsloth/Pixtral-12B-Base-2409-bnb-4bit",         # Pixtral base model
    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",          # Qwen2 VL support
    "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
    "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",
    "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",      # Any Llava variant works!
    "unsloth/llava-1.5-7b-hf-bnb-4bit",
] # More models at https://huggingface.co/unsloth

# 2mins
model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

==((====))==  Unsloth 2025.2.4: Fast Mllama vision patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/375k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

preprocessor_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

We now add LoRA adapters for parameter efficient finetuning - this allows us to only efficiently train 1% of all parameters.

**[NEW]** We also support finetuning ONLY the vision part of the model, or ONLY the language part. Or you can select both! You can also select to finetune the attention or the MLP layers!

In [ ]:
# 5s
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

Unsloth: Making `model.base_model.model.vision_model.transformer` require gradients


<a name="LLM Inference"></a>
### LLM INFERENCE
This let's us run the model for a few examples, so we can see what's happening as the model predicts inherently.

In [ ]:
FastVisionModel.for_inference(model) # Enable for inference!

image = dataset[1202]["image"]
instruction = "Classify the brain scan as Low Grade Glioma (0), High Grade Glioma (1), or No Glioma (2). Respond only in the following format: Choice: <0, 1, or 2> Reasoning: <Provide concise reasoning using 10 keywords based on the scan's visual features>."

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 20,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Choice: 0 Reasoning: Hypointensity, non-homogeneous, mild enhancement, per


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
8.537 GB of memory reserved.


In [ ]:
FastVisionModel.for_training(model) # Enable for training!

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer), # Must use!
    train_dataset = converted_dataset,
    args = SFTConfig(

        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,
        # num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate = 2e-6,
        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases

        # output_dir=main_data_path+"checkpoints", # Directory to save checkpoints
        # save_steps=30,                        # Save every 100 steps
        # save_total_limit=3,                    # Keep only the last 3 checkpoints
        # logging_dir=main_data_path+'logs',     # Directory for logging
        # logging_steps=1,                     # Log every 100 steps

        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        dataset_num_proc = 4,
        max_seq_length = 1024, # changed from 2048!
    ),
)

# trainer = SFTTrainer(
#     model=model,
#     tokenizer=tokenizer,
#     data_collator=UnslothVisionDataCollator(model, tokenizer),  # Must use!
#     train_dataset=converted_train_dataset,
#     eval_dataset=converted_val_dataset,  # Add validation dataset here
#     args=SFTConfig(
#         per_device_train_batch_size=16,
#         gradient_accumulation_steps=4,
#         warmup_steps=10,
#         num_train_epochs=1,  # Set this instead of max_steps for full training runs
#         learning_rate=2e-6,
#         fp16=not is_bf16_supported(),
#         bf16=is_bf16_supported(),
#         logging_steps=10,  # Log every 10 steps
#         evaluation_strategy="steps",  # Run validation during training
#         eval_steps=50,  # Evaluate every 50 steps
#         save_strategy="steps",  # Save checkpoints periodically
#         save_steps=50,  # Save every 50 steps
#         save_total_limit=3,  # Keep only the last 3 checkpoints
#         optim="adamw_8bit",
#         weight_decay=0.01,
#         lr_scheduler_type="linear",
#         seed=3407,
#         output_dir="outputs",
#         report_to="none",  # For Weights and Biases
#         remove_unused_columns=False,  # For vision fine-tuning
#         dataset_text_field="",
#         dataset_kwargs={"skip_prepare_dataset": True},
#         dataset_num_proc=4,
#         max_seq_length=1024,
#     ),
# )


In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Start the training
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 11,378 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 4
\        /    Total batch size = 64 | Total steps = 178
 "-____-"     Number of trainable parameters = 67,174,400
🦥 Unsloth needs about 1-3 minutes to load everything - please wait!


Step,Training Loss
1,3.901300
2,3.914900
3,3.910800
4,3.890900
5,3.900100
6,3.891800
7,3.888900
8,3.892300
9,3.897400
10,3.890500


In [ ]:
FastVisionModel.for_inference(model) # Enable for inference!

image = balanced_train_dataset[0]["image"]
ground_truth = balanced_train_dataset[0]["label"]
instruction = "Classify the brain scan as Low Grade Glioma (0), High Grade Glioma (1), or No Glioma (2) based on the scan's visual features. Respond only in the following format: Choice: <0, 1, or 2>."

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

print(f"Ground truth: {ground_truth}")
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 20,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Ground truth: 0
Choice: 0.<|eot_id|>


In [ ]:
model.save_pretrained("lora_full_model")
tokenizer.save_pretrained("lora_full_model")
model.push_to_hub("liufelic/lora_full_model", token = "token")
tokenizer.push_to_hub("liufelic/lora_full_model", token = "token")

README.md:   0%|          | 0.00/631 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

Saved model to https://huggingface.co/liufelic/lora_full_model


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:

# Save the model
model.save_pretrained("liufelic/Llama-3.2-11B-Vision-lora_full_model")
tokenizer.save_pretrained("liufelic/Llama-3.2-11B-Vision-lora_full_model")
model.push_to_hub("liufelic/Llama-3.2-11B-Vision-lora_full_model", token = "token")
tokenizer.push_to_hub("liufelic/Llama-3.2-11B-Vision-lora_full_model", token = "token")

# model.save_pretrained_merged("liufelic/Llama-3.2-11B-Vision-lora_full_model", tokenizer,)
# model.push_to_hub_merged("liufelic/Llama-3.2-11B-Vision-lora_full_model", tokenizer, save_method = "merged_16bit", token = "token")

README.md:   0%|          | 0.00/635 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

Saved model to https://huggingface.co/liufelic/Llama-3.2-11B-Vision-lora_full_model


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
# Save the model and tokenizer
model.save_pretrained("liufelic/Llama-3.2-11B-Vision-lora_16full_model")
tokenizer.save_pretrained("liufelic/Llama-3.2-11B-Vision-lora_16full_model")

# Save the model weights in float16
torch.save(model.state_dict(), "liufelic/Llama-3.2-11B-Vision-lora_16full_model/pytorch_model.bin", _use_new_zipfile_serialization=False)

# Push to Hugging Face Hub (optional)
model.push_to_hub("liufelic/Llama-3.2-11B-Vision-lora_16full_model", token = "token")
tokenizer.push_to_hub("liufelic/Llama-3.2-11B-Vision-lora_16full_model", token = "token

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
# Load model directly
# from transformers import AutoProcessor, AutoModelForImageTextToText
from transformers import MllamaForConditionalGeneration, AutoProcessor

model_id = "liufelic/Llama-3.2-11B-Vision-lora_16full_model"
model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto")
processor = AutoProcessor.from_pretrained(model_id)


# processor = AutoProcessor.from_pretrained("liufelic/Llama-3.2-11B-Vision-lora_full_model")
# model = AutoModelForImageTextToText.from_pretrained("liufelic/Llama-3.2-11B-Vision-lora_full_model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

image = balanced_train_dataset[0]["image"]
ground_truth = balanced_train_dataset[0]["label"]


instruction = "Classify the brain scan as Low Grade Glioma (0), High Grade Glioma (1), or No Glioma (2) based on the scan's visual features. Respond only in the following format: Choice: <0, 1, or 2>."

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]

input_text = processor.apply_chat_template(messages, add_generation_prompt=True)

# Prepare inputs
inputs = processor(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")

# inputs = {key: value.to(torch.float16) for key, value in inputs.items()}

# # Change aspect_ratio_ids to torch.long
# inputs['aspect_ratio_ids'] = inputs['aspect_ratio_ids'].type(torch.long)

# # Converting other tensors to float16 (if necessary)
# # Make sure to exclude 'aspect_ratio_ids' from the conversion
# for key, value in inputs.items():
#     if key != 'aspect_ratio_ids':  # Exclude aspect_ratio_ids from float16 conversion
#         inputs[key] = value.to(torch.float16)

# have the model generate a response
output = model.generate(**inputs, max_new_tokens=20)
response = processor.decode(output[0])

result_message = f"Ground truth: {ground_truth} || Model output: {response}"

print(result_message)

# FastVisionModel.for_inference(model) # Enable for inference!

# output_tokens = model.generate(**inputs, max_new_tokens=20,
#                                 use_cache=True, temperature=1.5, min_p=0.1)
# generated_text = extract_choice(tokenizer.decode(output_tokens[0], skip_special_tokens=True))





# print(f"Ground truth: {ground_truth}")
# text_streamer = TextStreamer(tokenizer, skip_prompt = True)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 20,
#                    use_cache = True, temperature = 1.5, min_p = 0.1)




RuntimeError: self and mat2 must have the same dtype, but got Half and Float

In [ ]:
import torch
    from transformers import MllamaForConditionalGeneration, AutoProcessor

    model_id = "decapoda-research/llama-7b-hf"  # Replace with your base model ID
    adapter_path = "lora_full_model"  # Replace with the path to your saved LoRA adapter

    # Load the base model in float16
    base_model = MllamaForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto",
    )

    # Load the LoRA adapter state dictionary
    lora_state_dict = torch.load(os.path.join(adapter_path, "adapter_model.bin"), map_location="cuda")

def find_lora_modules(model):
        lora_modules = []
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Linear) and name.endswith((".lora_A", ".lora_B")):
                lora_modules.append(module)
        return lora_modules

    lora_modules = find_lora_modules(base_model)

for lora_module in lora_modules:
        lora_name = lora_module.weight.data
        lora_module.weight.data = lora_state_dict[lora_name].type(torch.float16)
        # Assuming you saved the LoRA weights in float32, convert to float16

processor = AutoProcessor.from_pretrained(model_id)